In [5]:
from random import randint
from collections import defaultdict

import matplotlib.pyplot as plt
from ipynb.fs.full.Observer import CaptureMachine, TrackedObject, CaptureConfiguration, Camera, hStackImages, vStackImages
import dma.MechaCombat as mc
QuantumSystem = mc.QuantumSystem

In [ ]:
class QuantumMemory(TrackedObject):
    name: str

In [2]:
class HarmonyMachine(CaptureMachine):
    states = ["idle", "unstable", "illegal"]
    modes = ["passive", "calibrate", "add", "move", "action"]
    def __init__(self, captureConfiguration: CaptureConfiguration):
        super().__init__(captureConfiguration)
        self.hc = self.cc
    
    def isLegal(self, classifiedChange):
        return True
    
    def cycle(self):
        try:
            print(f"Starting Cycle {self.cycleCounter:5} -- {self}")
            self.cycleCounter += 1
            nextState = "idle"
            self.hc.capture()
            if self.mode == "passive":
                self.hc.setReference()
                return
            # else: mode in ["calibrate", "add", "move", "attack"]

            changes = self.referenceFrameDeltas()
            classification = None
            if changes.empty:
                self.hc.setReference()
            else:
                nextState = "unstable"
                if changes == self.lastChanges:  # TODO: and cameraChangeOverlap
                    classification = self.classifyChanges(changes)
                    if self.mode in ["move", "action"]:
                        try:
                            assert not classification.isNewObject(), f"Cannot add objects in {self.mode}"
                            assert self.isLegal(classification), f"{classificaton} is an illegal move"
                            nextState = "idle"
                            self.commitChanges(classification)
                            self.hc.setReference()
                        except AssertionError:
                            nextState = "illegal"
                    elif self.mode == "add":
                        nextState = "idle"
                        self.commitChanges(classification)
                        self.hc.setReference()
                    elif self.mode == "calibrate":
                        try:
                            self.calibrateToObject(classification)
                        except AssertionError as ae:
                            print(f"Failed Calibration: {ae}")
                        nextState = "idle"
                        self.hc.setReference()
                    else:
                        raise Exception("Unrecognized State")
            self.state = nextState
            self.cycleCounter += 1
            self.lastChanges = changes
        except:
            from traceback import format_exc
            print("CYCLE FAILURE!!!")
            print(format_exc())
            raise
    
    def actionMode(self):
        self.mode = "action"
    
    def moveMode(self):
        self.mode = "move"
    
    def addMode(self):
        self.mode = "add"
    
    def passiveMode(self):
        self.mode = "passive"
    
    def startCalibration(self):
        self.calibrationPts = []
        self.mode = "calibrate"
    
    def abortCalibration(self):
        self.calibrationPts = []
        self.mode = "passive"
            
    def cycleForChange(self):
        self.mode = "track"
        startLen = len(self.transitions)
        while len(self.transitions) == startLen:
            self.cycle()
        self.mode = "idle"
        return self.lastMemory
            
    def __repr__(self):
        return f"CapMac -- {self.mode} {str(len(self.calibrationPts)) + ' ' if self.mode == 'calibrate' else ''}{self.state}"

In [3]:
with open("harmonyConfiguration.json", "r") as f:
    config = f.read()
cc = CaptureConfiguration(config)
cameras = cc.cameras
cm = HarmonyMachine(cc)